Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

In [5]:
from helpers.vector_store import initialize_vector_store

initialize_vector_store(texts, "test", embedding, "hybrid")

Instantiate Elasticsearch Store

In [ ]:
from langchain_elasticsearch import ElasticsearchStore, BM25Strategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

bm25_store = ElasticsearchStore.from_documents(
  documents=texts,
  es_cloud_id=es_cloud_id,
  es_api_key=es_api_key,
  index_name="just_bm25",
  strategy=BM25Strategy()
)

Get LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

Get Retriever

In [ ]:
retriever = bm25_store.as_retriever(search_kwargs={"k": 5})

Create Chain

In [ ]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain, invoke_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [ ]:
question = "what are the interest rates for fixed deposit?"
result = invoke_conversational_retrieval_chain(chain, question)
result